In [ ]:
import os
from dotenv import load_dotenv

from typing import Final

import mlflow
import mlflow.entities
import mlflow.data.pandas_dataset
from mlflow.data.sources import (
    LocalArtifactDatasetSource
)

import pandas as pd
import numpy as np

from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import (
    MinMaxScaler,
    StandardScaler
)

import pickle

import PIL

In [ ]:
load_dotenv()

In [ ]:
INT8_MIN    = np.iinfo(np.int8).min
INT8_MAX    = np.iinfo(np.int8).max
INT16_MIN   = np.iinfo(np.int16).min
INT16_MAX   = np.iinfo(np.int16).max
INT32_MIN   = np.iinfo(np.int32).min
INT32_MAX   = np.iinfo(np.int32).max

FLOAT16_MIN = np.finfo(np.float16).min
FLOAT16_MAX = np.finfo(np.float16).max
FLOAT32_MIN = np.finfo(np.float32).min
FLOAT32_MAX = np.finfo(np.float32).max

def memory_usage(data, detail=1):
    if detail:
        display(data.memory_usage())
    memory = data.memory_usage().sum() / (1024*1024)
    print("Memory usage : {0:.2f}MB".format(memory))
    return memory

def compress_dataset(data):
    """
        Compress datatype as small as it can
        Parameters
        ----------
        path: pandas Dataframe

        Returns
        -------
            None
    """
    memory_before_compress = memory_usage(data, 0)
    print()
    length_interval      = 50
    length_float_decimal = 4

    print('='*length_interval)
    for col in data.columns:
        col_dtype = data[col][:100].dtype

        if col_dtype != 'object':
            print("Name: {0:24s} Type: {1}".format(col, col_dtype))
            col_series = data[col]
            col_min = col_series.min()
            col_max = col_series.max()

            if col_dtype == 'float64':
                print(" variable min: {0:15s} max: {1:15s}".format(str(np.round(col_min, length_float_decimal)), str(np.round(col_max, length_float_decimal))))
                if (col_min > FLOAT16_MIN) and (col_max < FLOAT16_MAX):
                    data[col] = data[col].astype(np.float16)
                    print("  float16 min: {0:15s} max: {1:15s}".format(str(FLOAT16_MIN), str(FLOAT16_MAX)))
                    print("compress float64 --> float16")
                elif (col_min > FLOAT32_MIN) and (col_max < FLOAT32_MAX):
                    data[col] = data[col].astype(np.float32)
                    print("  float32 min: {0:15s} max: {1:15s}".format(str(FLOAT32_MIN), str(FLOAT32_MAX)))
                    print("compress float64 --> float32")
                else:
                    pass
                memory_after_compress = memory_usage(data, 0)
                print("Compress Rate: [{0:.2%}]".format((memory_before_compress-memory_after_compress) / memory_before_compress))
                print('='*length_interval)

            if col_dtype == 'int64':
                print(" variable min: {0:15s} max: {1:15s}".format(str(col_min), str(col_max)))
                type_flag = 64
                if (col_min > INT8_MIN/2) and (col_max < INT8_MAX/2):
                    type_flag = 8
                    data[col] = data[col].astype(np.int8)
                    print("     int8 min: {0:15s} max: {1:15s}".format(str(INT8_MIN), str(INT8_MAX)))
                elif (col_min > INT16_MIN) and (col_max < INT16_MAX):
                    type_flag = 16
                    data[col] = data[col].astype(np.int16)
                    print("    int16 min: {0:15s} max: {1:15s}".format(str(INT16_MIN), str(INT16_MAX)))
                elif (col_min > INT32_MIN) and (col_max < INT32_MAX):
                    type_flag = 32
                    data[col] = data[col].astype(np.int32)
                    print("    int32 min: {0:15s} max: {1:15s}".format(str(INT32_MIN), str(INT32_MAX)))
                    type_flag = 1
                else:
                    pass
                memory_after_compress = memory_usage(data, 0)
                print("Compress Rate: [{0:.2%}]".format((memory_before_compress-memory_after_compress) / memory_before_compress))
                if type_flag == 32:
                    print("compress (int64) ==> (int32)")
                elif type_flag == 16:
                    print("compress (int64) ==> (int16)")
                else:
                    print("compress (int64) ==> (int8)")
                print('='*length_interval)

    print()
    memory_after_compress = memory_usage(data, 0)
    print("Compress Rate: [{0:.2%}]".format((memory_before_compress-memory_after_compress) / memory_before_compress))

In [ ]:
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("TRACKING_USER")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("TRACKING_PSWD")
os.environ["MLFLOW_HTTP_REQUEST_TIMEOUT"] = "9000"

In [ ]:
mlflow.set_tracking_uri(f"http://{os.getenv('ADRESS')}:{os.getenv('PORT')}")
CURRENT_EXPERIMENT: mlflow.entities.Experiment = mlflow.set_experiment("Spectrum_EDA")

In [ ]:
# DATA SAMPLE, SPECTRUM
# ----------
SAMPLE_PATH: str = "../data/raw/spectr/Выборка_Н22_5_1.csv"
SAMPLE_REST: str = "../data/raw/spectr/СКЗУ_ВЫБОРКА_ЯНВАР-МАРТ.csv"
# HISTORICAL DATA
# ----------
RMSA_PATH: str = "../data/raw/historical/СКЗУ.csv"
RMSA10_PATH: str = "../data/raw/historical/СКЗУ10.csv"
RMSA_GROWTH: str = "../data/raw/historical/СКЗУ_РОСТ.csv"
RMSA_AMP: str = "../data/raw/historical/СКЗУ_АМПЛИТУДА.csv"
RMSD_SPAN: str = "../data/raw/historical/СКЗП_РАЗМАХ.csv"
PK_FACTOR: str = "../data/raw/historical/ПИК_ФАКТОР.csv"

df = pd.read_csv(SAMPLE_PATH, skiprows=16, sep=';')
df.drop(columns=df.columns[[0, 1]], inplace=True)

In [ ]:
# Dataset w/o dates
column_names = pd.to_datetime(df.iloc[0, ::2].tolist(), format='%d.%m.%Y %H:%M:%S') 
df = df.iloc[:, 1::2]
df.columns = column_names
df = df.sort_index(axis=1)

In [ ]:
# Values to float
df = df.apply(lambda x:
              pd.to_numeric(
                  x.str.replace(',','.'),
                  errors='coerce')
              )

Log raw dataset

In [ ]:
# IF CHANGES MADE CHANGE FILENAME: SAMPLE_1, SAMPLE_2, ...
SAMPLE_LOG_PATH: Final[str] = "../data/cleaned/spectr/sample.csv"
# TO LOG ONLY ONCE
if not os.path.isfile(SAMPLE_LOG_PATH):
    df.to_csv("../data/cleaned/spectr/sample.csv", index=False)
else:
    # IMPLEMENT A COMPARISON OF 2 VERSIONS OF DATASET
    ...

In [ ]:
df.describe().to_html("./artifacts/stats.html")

In [ ]:
# DF columns to string to log dataset
df.columns = df.columns.astype(str)

RUN_NAME: str = "Sample dataset"

# check if experiment contains run with such name
# if so, then change run name (manually) and log dataset
# else log dataset

# GET ALL RUNS OF EXPERIMENT
# ----------
runs = mlflow.search_runs([CURRENT_EXPERIMENT.experiment_id])
# CHECK IF EXPERIMENT 
# ----------
if runs[runs["tags.mlflow.runName"] == RUN_NAME].shape[0] == 0:
    dataset = mlflow.data.pandas_dataset.from_pandas(
        df, 
        name="Raw Samples",
        source=""
    )

    with mlflow.start_run(run_name=RUN_NAME) as run:
        # Log dataset stats
        mlflow.log_artifact(local_path="./artefacts/stats.html")
        # Log dataset fields info
        mlflow.log_input(dataset, context="EDA")

    RUN_ID = run.info.run_id
    logged_run = mlflow.get_run(run.info.run_id)

    # Retrieve the Dataset object
    logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

    # View some of the recorded Dataset information
    print(f"Dataset name: {logged_dataset.name}")
    print(f"Dataset digest: {logged_dataset.digest}")
    print(f"Dataset profile: {logged_dataset.profile}")
    print(f"Dataset schema: {logged_dataset.schema}")
else:
    # IMPLEMENT CHANGING OF RUN NAME
    ...

In [ ]:
compress_dataset(df)

In [ ]:
# IF CHANGES MADE CHANGE FILENAME: SAMPLE_1, SAMPLE_2, ...
SAMPLE_COMPRESSED_LOG_PATH: Final[str] = "../data/processed/spectr/compressed.csv"
# TO LOG ONLY ONCE
if not os.path.isfile(SAMPLE_COMPRESSED_LOG_PATH):
    df.to_csv(SAMPLE_COMPRESSED_LOG_PATH, index=False)
else:
    # IMPLEMENT A COMPARISON OF 2 VERSIONS OF DATASET
    ...

In [ ]:
# DF columns to string to log dataset
df.columns = df.columns.astype(str)

RUN_NAME: str = "Compressed dataset"

# check if experiment contains run with such name
# if so, then change run name (manually) and log dataset
# else log dataset

# GET ALL RUNS OF EXPERIMENT
# ----------
runs = mlflow.search_runs([CURRENT_EXPERIMENT.experiment_id])
# CHECK IF EXPERIMENT EXISTS
# ----------
if runs[runs["tags.mlflow.runName"] == RUN_NAME].shape[0] == 0:
    dataset = mlflow.data.pandas_dataset.from_pandas(
        df, 
        name="Compressed types",
        source=LocalArtifactDatasetSource("C:/Users/egore/Desktop/predictive_RUL/data/processed/spectr/compressed.csv")
    )

    with mlflow.start_run(run_name=RUN_NAME) as run:
        # Log dataset fields info
        mlflow.log_input(dataset, context="COMPRESSED_DATA_EDA")


    RUN_ID = run.info.run_id
    logged_run = mlflow.get_run(run.info.run_id)

    # Retrieve the Dataset object
    logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

    # View some of the recorded Dataset information
    print(f"Dataset name: {logged_dataset.name}")
    print(f"Dataset digest: {logged_dataset.digest}")
    print(f"Dataset profile: {logged_dataset.profile}")
    print(f"Dataset schema: {logged_dataset.schema}")
else:
    # IMPLEMENT CHANGING OF RUN NAME
    ...

In [ ]:
dates = df.T.index
feature_dataset = pd.DataFrame()
feature_dataset["kurtosis"] = df.apply(lambda x: stats.kurtosis(x.astype(np.float64)))
feature_dataset["skewness"] = df.apply(lambda x: stats.skew(x.astype(np.float64)))
feature_dataset["peak2peak"] = df.apply(lambda x: np.ptp(x.astype(np.float64)))
feature_dataset["mean"] = df.apply(lambda x: x.astype(np.float64).mean())
feature_dataset["std"] = df.apply(lambda x: x.astype(np.float64).std())
feature_dataset["shapeFactor"] = df.apply(lambda x: np.sqrt(np.mean(np.square(x.astype(np.float64)))) / np.abs(x.astype(np.float64).mean()))
feature_dataset["energy"] = df.apply(lambda x: np.sum(x.astype(np.float64) ** 2))
feature_dataset.index = dates

In [ ]:
# IF CHANGES MADE CHANGE FILENAME: SAMPLE_1, SAMPLE_2, ...
SAMPLE_FEATURE_LOG_PATH: Final[str] = "../data/processed/spectr/feature_dataset.csv"
# TO LOG ONLY ONCE
if not os.path.isfile(SAMPLE_FEATURE_LOG_PATH):
    feature_dataset.to_csv(SAMPLE_FEATURE_LOG_PATH, index=False)
else:
    # IMPLEMENT A COMPARISON OF 2 VERSIONS OF DATASET
    ...

In [ ]:
# run id used, cuz i fucked up run names
RUN_ID = "3fbc94286d0a4bf889137e5dc99704ca"
with mlflow.start_run(run_id=RUN_ID) as run:
    fig, ax = plt.subplots(7, figsize=(16,16))
    for i, col in enumerate(feature_dataset.columns):
        sns.lineplot(feature_dataset[col], ax=ax[i])
        ax[i].set_xticklabels("")
    plt.savefig("./artifacts/feature_dataset.png")
    mlflow.log_figure(fig, "./artifacts/feature_dataset.png")

Normilize

In [ ]:
with mlflow.start_run(run_id="3fbc94286d0a4bf889137e5dc99704ca") as run:
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(feature_dataset)

    with open("./artifacts/scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)

    mlflow.log_artifact("./artifacts/scaler.pkl", "artifacts/utils/scaler.pkl")

Find out the number of components

In [ ]:
with mlflow.start_run(run_id="3fbc94286d0a4bf889137e5dc99704ca") as run:
    pca = PCA().fit(scaled_features)
    fig, ax = plt.subplots()
    ax.plot(np.cumsum(pca.explained_variance_ratio_))
    ax.axhline(0.95, c="red")
    ax.set_xlabel('number of components')
    ax.set_ylabel('cumulative explained variance')
    ax.set_title("Number of components vs explained variance")

    plt.savefig("./artifacts/pca_var.png")

    mlflow.log_figure(fig, "artifacts/pca_var.png")

    with open("./artifacts/pca.pkl", "wb") as f:
        pickle.dump(pca, f)
    
    mlflow.log_artifact("./artifacts/pca.pkl", "artifacts/utils/pca.pkl")


* Create new run
* Log feature dataset  
* log scaled dataset  
* log pca3 & pca4

In [ ]:
runs[runs["tags.mlflow.runName"] == RUN_NAME]['run_id'][0]

In [ ]:
# DF columns to string to log dataset
feature_dataset.columns = feature_dataset.columns.astype(str)

RUN_NAME: str = "PCA_SEACH_INDICATOR"

# GET ALL RUNS OF EXPERIMENT
# ----------
runs = mlflow.search_runs([CURRENT_EXPERIMENT.experiment_id])
# CHECK IF EXPERIMENT EXISTS
# ----------
dataset = mlflow.data.pandas_dataset.from_pandas(
    feature_dataset, 
    name="FEATURE DATASET",
    source=""
)
if runs[runs["tags.mlflow.runName"] == RUN_NAME].shape[0] == 0:
    with mlflow.start_run(run_name=RUN_NAME) as run:
        # Log dataset fields info
        mlflow.log_input(dataset, context="SEARCH HEALTH INDICATOR")
        # Log dataset csv
        mlflow.log_artifact("../data/processed/spectr/feature_dataset.csv", "artifacts/data/feature_dataset.csv")
        # Log dataset as JSON
        mlflow.log_table(feature_dataset, "artifacts/data_json/feature_dataset.json")
else:
    with mlflow.start_run(run_id=runs[runs["tags.mlflow.runName"] == RUN_NAME]['run_id'][0]):
        ...

RUN_ID = run.info.run_id
logged_run = mlflow.get_run(run.info.run_id)

# Retrieve the Dataset object
logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

# View some of the recorded Dataset information
print(f"Dataset name: {logged_dataset.name}")
print(f"Dataset digest: {logged_dataset.digest}")
print(f"Dataset profile: {logged_dataset.profile}")
print(f"Dataset schema: {logged_dataset.schema}")
    

Test 3 and 4 components (create 2 runs)

In [ ]:
with mlflow.start_run(run_id="36d231c830034e888ad1e04dd67741d5") as run:
    pca = PCA(n_components=3)
    pca3 = pca.fit_transform(scaled_features)

    fig = sns.pairplot(pd.DataFrame(pca3), corner=True, kind="scatter")

    plt.savefig("./artifacts/pairplot_pca.png")

    mlflow.log_image(PIL.Image.open("./artifacts/pairplot_pca.png"), "artifacts/charts/pairplot_pca.png")

In [ ]:
with mlflow.start_run(run_id="36d231c830034e888ad1e04dd67741d5") as run:
    fig, ax = plt.subplots()
    sns.scatterplot(x=pca3[:, 0], y=pca3[:, 1], hue=dates[:], legend=False, ax=ax)
    plt.savefig("./artifacts/pca1_pca2")
    mlflow.log_figure(fig, "artifacts/charts/pca1_pca2.png")
    plt.figure()
    sns.scatterplot(x=pca3[:, 0], y=pca3[:, 2], hue=dates[:], legend=False, ax=ax)
    plt.savefig("./artifacts/pca1_pca3")
    mlflow.log_figure(fig, "artifacts/charts/pca1_pca3.png")
    plt.figure()
    sns.scatterplot(x=pca3[:, 1], y=pca3[:, 2], hue=dates[:], legend=False, ax=ax)
    plt.savefig("./artifacts/pca2_pca3")
    mlflow.log_figure(fig, "artifacts/charts/pca2_pca3.png")

In [ ]:
with mlflow.start_run(run_id="36d231c830034e888ad1e04dd67741d5") as run:
    pca = PCA(n_components=4)
    pca4 = pca.fit_transform(scaled_features)
    sns.pairplot(pd.DataFrame(pca4), corner=True, kind="scatter")

    fig = sns.pairplot(pd.DataFrame(pca4), corner=True, kind="scatter")

    plt.savefig("./artifacts/pairplot_pca4.png")

    mlflow.log_image(PIL.Image.open("./artifacts/pairplot_pca4.png"), "artifacts/charts/pairplot_pca4.png")

READING REST OF THE SAMPLE DATASETS

In [ ]:
df_rest = pd.read_csv(SAMPLE_REST, skiprows=16, sep=';')
df_rest.drop(columns=df_rest.columns[[0, 1]], inplace=True)

column_names = pd.to_datetime(df_rest.iloc[0, ::2].tolist(), format='%d.%m.%Y %H:%M:%S') 
df_rest = df_rest.iloc[:, 1::2]
df_rest.columns = column_names
df_rest = df_rest.sort_index(axis=1)

# Values to float
df_rest = df_rest.apply(lambda x:
              pd.to_numeric(
                  x.str.replace(',','.'),
                  errors='coerce')
              )

compress_dataset(df_rest)

In [ ]:
dates_rest = df_rest.T.index
feat_dataset_rest = pd.DataFrame()
feat_dataset_rest["kurtosis"] = df_rest.apply(lambda x: stats.kurtosis(x.astype(np.float64)))
feat_dataset_rest["skewness"] = df_rest.apply(lambda x: stats.skew(x.astype(np.float64)))
feat_dataset_rest["peak2peak"] = df_rest.apply(lambda x: np.ptp(x.astype(np.float64)))
feat_dataset_rest["mean"] = df_rest.apply(lambda x: x.astype(np.float64).mean())
feat_dataset_rest["std"] = df_rest.apply(lambda x: x.astype(np.float64).std())
feat_dataset_rest["shapeFactor"] = df_rest.apply(lambda x: np.sqrt(np.mean(np.square(x.astype(np.float64)))) / np.abs(x.astype(np.float64).mean()))
feat_dataset_rest["energy"] = df_rest.apply(lambda x: np.sum(x.astype(np.float64) ** 2))
feat_dataset_rest.index = dates_rest

In [ ]:
feat_dataset_rest.to_csv("../data/processed/spectr/feature_dataset_rest.csv")

In [ ]:
# DF columns to string to log dataset
feat_dataset_rest.columns = feat_dataset_rest.columns.astype(str)

RUN_NAME: str = "PCA_SEACH_INDICATOR"

# GET ALL RUNS OF EXPERIMENT
# ----------
runs = mlflow.search_runs([CURRENT_EXPERIMENT.experiment_id])
# CHECK IF EXPERIMENT EXISTS
# ----------
dataset = mlflow.data.pandas_dataset.from_pandas(
    feat_dataset_rest, 
    name="FEATURE DATASET JANUARY-MARCH",
    source=""
)
if runs[runs["tags.mlflow.runName"] == RUN_NAME].shape[0] == 0:
    with mlflow.start_run(run_name=RUN_NAME) as run:
        # Log dataset fields info
        mlflow.log_input(dataset, context="SEARCH HEALTH INDICATOR")
        # Log dataset csv
        mlflow.log_artifact("../data/processed/spectr/feature_dataset.csv", "artifacts/data/feature_dataset.csv")
        # Log dataset as JSON
        mlflow.log_table(feature_dataset, "artifacts/data_json/feature_dataset.json")
else:
    with mlflow.start_run(run_id=runs[runs["tags.mlflow.runName"] == RUN_NAME]['run_id'][0]):
        # Log dataset fields info
        mlflow.log_input(dataset, context="READ 6 MONTH RANGE")
        # Log dataset csv
        mlflow.log_artifact("../data/processed/spectr/feature_dataset_rest.csv", "artifacts/data/feature_dataset_rest.csv")
        # Log dataset as JSON
        mlflow.log_table(feat_dataset_rest, "artifacts/data_json/feature_dataset_rest.json")

RUN_ID = run.info.run_id
logged_run = mlflow.get_run(run.info.run_id)

# Retrieve the Dataset object
logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

# View some of the recorded Dataset information
print(f"Dataset name: {logged_dataset.name}")
print(f"Dataset digest: {logged_dataset.digest}")
print(f"Dataset profile: {logged_dataset.profile}")
print(f"Dataset schema: {logged_dataset.schema}")
    

In [ ]:
feat_dataset_full = pd.concat([feature_dataset, feat_dataset_rest])

In [ ]:
feat_dataset_full.to_csv("../data/processed/spectr/feature_dataset_full.csv")

In [ ]:
# DF columns to string to log dataset
feat_dataset_full.columns = feat_dataset_full.columns.astype(str)

RUN_NAME: str = "PCA_SEACH_INDICATOR"

# GET ALL RUNS OF EXPERIMENT
# ----------
runs = mlflow.search_runs([CURRENT_EXPERIMENT.experiment_id])
# CHECK IF EXPERIMENT EXISTS
# ----------
dataset = mlflow.data.pandas_dataset.from_pandas(
    feat_dataset_full, 
    name="FEATURE DATASET JANUARY-MARCH",
    source=""
)
if runs[runs["tags.mlflow.runName"] == RUN_NAME].shape[0] == 0:
    with mlflow.start_run(run_name=RUN_NAME) as run:
        # Log dataset fields info
        mlflow.log_input(dataset, context="SEARCH HEALTH INDICATOR")
        # Log dataset csv
        mlflow.log_artifact("../data/processed/spectr/feature_dataset.csv", "artifacts/data/feature_dataset.csv")
        # Log dataset as JSON
        mlflow.log_table(feat_dataset_full, "artifacts/data_json/feature_dataset.json")
else:
    with mlflow.start_run(run_id=runs[runs["tags.mlflow.runName"] == RUN_NAME]['run_id'][0]):
        # Log dataset fields info
        mlflow.log_input(dataset, context="CONCAT 2 PART OF DATASET")
        # Log dataset csv
        mlflow.log_artifact("../data/processed/spectr/feature_dataset_full.csv", "artifacts/data/feature_dataset_full.csv")
        # Log dataset as JSON
        mlflow.log_table(feat_dataset_full, "artifacts/data_json/feature_dataset_full.json")

RUN_ID = run.info.run_id
logged_run = mlflow.get_run(run.info.run_id)

# Retrieve the Dataset object
logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

# View some of the recorded Dataset information
print(f"Dataset name: {logged_dataset.name}")
print(f"Dataset digest: {logged_dataset.digest}")
print(f"Dataset profile: {logged_dataset.profile}")
print(f"Dataset schema: {logged_dataset.schema}")
    

In [ ]:
with mlflow.start_run(run_id="36d231c830034e888ad1e04dd67741d5") as run:
    fig, ax = plt.subplots(figsize=(16,16))
    feat_dataset_full.plot(subplots=True, ax=ax)

    mlflow.log_figure(fig, "artifacts/charts/full_signals.png")


In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(feat_dataset_full)

pca = PCA().fit(scaled_features)
fig, ax = plt.subplots()
ax.plot(np.cumsum(pca.explained_variance_ratio_))
ax.axhline(0.95, c="green")
ax.axvspan(3, 4, alpha=.5, color="red")
ax.set_xlabel('number of components')
ax.set_ylabel('cumulative explained variance')
ax.set_title("Number of components vs explained variance")

with mlflow.start_run(run_id="36d231c830034e888ad1e04dd67741d5") as run:
    mlflow.log_figure(fig, "artifacts/charts/components_vs_variance.png")

In [ ]:
with mlflow.start_run(run_id="36d231c830034e888ad1e04dd67741d5") as run:
    pca = PCA(n_components=4)
    pca4 = pca.fit_transform(scaled_features)

    fig = sns.pairplot(pd.DataFrame(pca4), corner=True, kind="scatter")

    plt.savefig("./artifacts/pairplot_pca_full.png")

    mlflow.log_image(PIL.Image.open("./artifacts/pairplot_pca_full.png"), "artifacts/charts/pairplot_pca_full.png")